### Part 1.6 - Extract_data_30000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Sixth 5000 movies (30,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the sixth 5000 movies and create the souplist of those URLs
"""
content_six = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_six_13012020.pkl'), 'rb') as f:
    
    content_six = pickle.load(f)

print("Number of URLs: {}".format(len(content_six)))

content_souplist_six = []

for i in tqdm_notebook(content_six):
    
    content_souplist_six.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_six)))

In [ ]:
"""
Open the synopsis urls of the sixth 5000 movies and create the souplist of those URLs
"""
synopsis_six = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_six_13012020.pkl'), 'rb') as f:
    
    synopsis_six = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_six)))

synopsis_souplist_six = []

for i in tqdm_notebook(synopsis_six):
    
    synopsis_souplist_six.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_six)))

In [ ]:
"""
Open the reviews urls of the sixth 5000 movies and create the souplist of those URLs
"""
review_six = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_six_15012020.pkl'), 'rb') as f:
    
    review_six = pickle.load(f)

print("Number of URLs: {}".format(len(review_six)))

review_souplist_six = []

for i in tqdm_notebook(review_six):
    
    review_souplist_six.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_six)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_six)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_six)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_six)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_six)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_six)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_six)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_six_13012020.pkl'), 'rb') as f:
    
    content_six = pickle.load(f)

print("Number of URLs: {}".format(len(content_six)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_six_15012020.pkl'), 'rb') as f:
    
    review_six = pickle.load(f)

print("Number of URLs: {}".format(len(review_six)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\pre-indexed files\\plot_six_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\pre-indexed files\\rating_six_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\pre-indexed files\\actors_six_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\pre-indexed files\\director_six_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\pre-indexed files\\synopsis_six_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\pre-indexed files\\reviews_six_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[45,56,72,74,169,313,532,585,689,805,838,977,1041,1042,1043,1157,1158,1159,1160,1161,1164,1165,1168,1170,1171,1214,1228,1229,1258,1442,1445,1491,1670,1671,1678,1706,1995,2078,2217,2622,2783,3248,3349,3522,3523,3784,3846,3851,3853,3863,3871,3890,3900,3908,3912,3913,3918,3919,3923,3929,3932,4018,4027,4038,4062,4122,4136,4142,4145,4153,4165,4166,4188,4192,4193,4206,4241,4248,4256,4352,4354,4355,4356,4437,4607,4733,4808,4809,4852,4883,1987,2551,3482,3981,4887,68,176,177,178,179,180,299,309,391,509,528,531,585,658,815,977,1001,1088,1099,1109,1156,1178,1180,1185,1190,1195,1197,1198,1210,1229,1396,1438,1451,1629,1676,1698,1706,1944,1964,2002,2021,2043,2618,2743,3482,3800,3819,3838,3896,4038,4099,4147,4179,4356,4446,4496,4575,4576,4592,4606,4829,4846,4855,4887,4894,4952,4986,4990]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_six = [i for j, i in enumerate(content_six) if j not in index_remove]
review_index_six = [i for j, i in enumerate(review_six) if j not in index_remove]
print(len(content_index_six))
print(len(review_index_six))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_six\\content_index_six_20012020.pkl'), 'wb') as f: 
#     pickle.dump(content_index_six, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_six\\review_index_six_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_six, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_six = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\content_index_six_20012020.pkl'), 'rb') as f:
    
    content_index_six = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_six)))

for i in tqdm_notebook(content_index_six):
    
    content_souplist_six.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_six)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\review_index_six_20012020.pkl'), 'rb') as f:
    
    review_six = pickle.load(f)

print("Number of URLs: {}".format(len(review_six)))

review_souplist_six = []

for i in tqdm_notebook(review_six):
    
    review_souplist_six.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_six)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_six)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_six)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_six)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_six)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_six)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\plot_six_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\actors_six_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\director_six_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\reviews_six_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_six, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\rating_six_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[45,56,72,74,169,313,532,585,689,805,838,977,1041,1042,1043,1157,1158,1159,1160,1161,1164,1165,1168,1170,1171,1214,1228,1229,1258,1442,1445,1491,1670,1671,1678,1706,1995,2078,2217,2622,2783,3248,3349,3522,3523,3784,3846,3851,3853,3863,3871,3890,3900,3908,3912,3913,3918,3919,3923,3929,3932,4018,4027,4038,4062,4122,4136,4142,4145,4153,4165,4166,4188,4192,4193,4206,4241,4248,4256,4352,4354,4355,4356,4437,4607,4733,4808,4809,4852,4883,1987,2551,3482,3981,4887,68,176,177,178,179,180,299,309,391,509,528,531,585,658,815,977,1001,1088,1099,1109,1156,1178,1180,1185,1190,1195,1197,1198,1210,1229,1396,1438,1451,1629,1676,1698,1706,1944,1964,2002,2021,2043,2618,2743,3482,3800,3819,3838,3896,4038,4099,4147,4179,4356,4446,4496,4575,4576,4592,4606,4829,4846,4855,4887,4894,4952,4986,4990]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_six = dataset.iloc[25000:30000].reset_index(drop=True)
dataset_six = dataset_six[~dataset_six.index.isin(index_remove)]
dataset_six.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the sixth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\plot_six_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\rating_six_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\actors_six_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\director_six_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_six\\reviews_six_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_six['actors'] = actors
dataset_six['plot'] = plot
dataset_six['imdb_rating'] = rating
dataset_six['director'] = director
dataset_six['reviews'] = reviews
dataset_six = dataset_six.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[8,29,91,92,132,143,171,186,249,286,299,300,321,329,337,406,435,448,457,459,491,506,511,513,562,631,639,649,650,669,672,693,697,783,787,788,793,800,808,833,861,862,870,907,933,976,984,992,995,996,999,1033,1038,1103,1135,1146,1148,1150,1151,1155,1159,1161,1168,1191,1221,1258,1261,1317,1333,1335,1347,1379,1493,1536,1538,1540,1543,1566,1570,1586,1606,1608,1617,1621,1623,1631,1648,1666,1674,1780,1844,1867,1885,1914,1956,1972,1996,1998,2026,2047,2073,2080,2101,2125,2134,2157,2317,2328,2330,2366,2492,2535,2536,2572,2677,2814,2836,2865,2867,2928,2936,3082,3147,3154,3217,3229,3310,3364,3395,3647,3691,3695,3697,3707,3714,3718,3746,3751,3758,3769,3792,3794,3796,3797,3804,3817,3818,3827,3829,3837,3851,3852,3861,3924,3931,3944,3956,3957,3972,3979,4007,4010,4029,4031,4039,4040,4041,4057,4078,4083,4117,4146,4151,4165,4189,4214,4215,4221,4234,4299,4309,4323,4357,4404,4445,4448,4451,4468,4472,4486,4489,4523,4525,4527,4529,4533,4546,4552,4577,4579,4597,4603,4625,4644,4698,4716,4768,4798,4800,4836]
dataset_six = dataset_six[~dataset_six.index.isin(index_remove)]
dataset_six = dataset_six[dataset_six.astype(str)['reviews'] != '[]']
dataset_six.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_six.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_six\\dataset_six_final_25012020.pkl"))